# Flight Price Prediction
### https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction

In [2]:
# Imports
import pandas as pd 
import numpy as np
import os 

In [3]:
# List all files within data/raw
raw_data_folder = "../data/raw"
os.listdir(raw_data_folder)

['.gitkeep', 'business.csv', 'economy.csv']

In [4]:
# Load raw data 
raw_data_business = pd.read_csv(raw_data_folder+"/business.csv")
raw_data_economy = pd.read_csv(raw_data_folder+"/economy.csv")

In [5]:
print('Shape of raw data business: {0}'.format(raw_data_business.shape))
print('Shape of raw data economy: {0}'.format(raw_data_economy.shape))

Shape of raw data business: (93487, 11)
Shape of raw data economy: (206774, 11)


#### Lets do high level EDA and preprocessing along the way 
1. Concat 2 dataframes
2. Check missing values
3. Check duplicate values
4. Check data types
5. Check unique values
6. Check stats
7. Check each column and clean if necessary
8. Export preprocessed data to ./data/processed

In [6]:
raw_data_business.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,Air India,AI,868,18:00,Delhi,02h 00m,non-stop,20:00,Mumbai,"25,612"
1,11-02-2022,Air India,AI,624,19:00,Delhi,02h 15m,non-stop,21:15,Mumbai,"25,612"
2,11-02-2022,Air India,AI,531,20:00,Delhi,24h 45m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:45,Mumbai,"42,220"
3,11-02-2022,Air India,AI,839,21:25,Delhi,26h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"44,450"
4,11-02-2022,Air India,AI,544,17:15,Delhi,06h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,23:55,Mumbai,"46,690"


In [7]:
raw_data_economy.head()

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price
0,11-02-2022,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,"5,953"
1,11-02-2022,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,"5,953"
2,11-02-2022,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,"5,956"
3,11-02-2022,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,"5,955"
4,11-02-2022,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,"5,955"


##### Concat 2 raw datasets

In [8]:
# Add an identifier 
raw_data_economy['type'] = 'Economy'
raw_data_business['type'] = 'Business'

raw_data = pd.concat([raw_data_economy, raw_data_business], ignore_index=True)

In [9]:
raw_data['type'].value_counts()

type
Economy     206774
Business     93487
Name: count, dtype: int64

##### Check missing values

In [10]:
raw_data.isna().sum()

date          0
airline       0
ch_code       0
num_code      0
dep_time      0
from          0
time_taken    0
stop          0
arr_time      0
to            0
price         0
type          0
dtype: int64

##### Check duplicate rows

In [11]:
raw_data[raw_data.duplicated()]

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,type
563,14-02-2022,Air India,AI,807,17:20,Delhi,15h 15m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,08:35,Mumbai,"12,150",Economy
6181,13-03-2022,Air India,AI,475,13:00,Delhi,24h 35m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,13:35,Mumbai,"4,780",Economy


In [12]:
# Delete the duplicated rows
print('Shape before dropping duplicates: {0}'.format(raw_data.shape))
raw_data.drop_duplicates(inplace=True, ignore_index=True)
print('Shape after dropping duplicates: {0}'.format(raw_data.shape))

Shape before dropping duplicates: (300261, 12)
Shape after dropping duplicates: (300259, 12)


##### Check data types

In [13]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300259 entries, 0 to 300258
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        300259 non-null  object
 1   airline     300259 non-null  object
 2   ch_code     300259 non-null  object
 3   num_code    300259 non-null  int64 
 4   dep_time    300259 non-null  object
 5   from        300259 non-null  object
 6   time_taken  300259 non-null  object
 7   stop        300259 non-null  object
 8   arr_time    300259 non-null  object
 9   to          300259 non-null  object
 10  price       300259 non-null  object
 11  type        300259 non-null  object
dtypes: int64(1), object(11)
memory usage: 27.5+ MB


In [14]:
# Convert 'date' to datetime 

print("Type of 'date' before: {0}".format(type(raw_data['date'][0])))
raw_data['date'] = pd.to_datetime(raw_data['date'], format="%d-%m-%Y")
print("Type of 'date' after: {0}\n".format(type(raw_data['date'][0])))

# Convert 'num_code' to string 

print("Type of 'num_code' before: {0}".format(type(raw_data['num_code'][0])))
raw_data['num_code'] = raw_data['num_code'].apply(str)
print("Type of 'num_code' after: {0}\n".format(type(raw_data['num_code'][0])))

# Convert 'price' to int

print("Type of 'price' before: {0}".format(type(raw_data['price'][0])))
raw_data['price'] = pd.to_numeric(raw_data['price'].apply(lambda x: x.replace(",","")))
print("Type of 'price' after: {0}\n".format(type(raw_data['price'][0])))

Type of 'date' before: <class 'str'>
Type of 'date' after: <class 'pandas._libs.tslibs.timestamps.Timestamp'>

Type of 'num_code' before: <class 'numpy.int64'>
Type of 'num_code' after: <class 'str'>

Type of 'price' before: <class 'str'>
Type of 'price' after: <class 'numpy.int64'>



In [15]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300259 entries, 0 to 300258
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        300259 non-null  datetime64[ns]
 1   airline     300259 non-null  object        
 2   ch_code     300259 non-null  object        
 3   num_code    300259 non-null  object        
 4   dep_time    300259 non-null  object        
 5   from        300259 non-null  object        
 6   time_taken  300259 non-null  object        
 7   stop        300259 non-null  object        
 8   arr_time    300259 non-null  object        
 9   to          300259 non-null  object        
 10  price       300259 non-null  int64         
 11  type        300259 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 27.5+ MB


##### Check unique values

In [16]:
raw_data.nunique().sort_values(ascending=False)

price         12165
num_code       1255
time_taken      483
arr_time        266
dep_time        251
date             49
stop             40
airline           8
ch_code           8
from              6
to                6
type              2
dtype: int64

##### Check stats

In [17]:
raw_data.describe()

,date,price
count,300259,300259.000000
mean,2022-03-08 00:06:31.342141440,20883.800386
min,2022-02-11 00:00:00,1105.000000
25%,2022-02-25 00:00:00,4783.000000
50%,2022-03-08 00:00:00,7425.000000
75%,2022-03-20 00:00:00,42521.000000
max,2022-03-31 00:00:00,123071.000000
std,NaN,22695.962230


##### Check each column and clean if necessary

In [18]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300259 entries, 0 to 300258
Data columns (total 12 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   date        300259 non-null  datetime64[ns]
 1   airline     300259 non-null  object        
 2   ch_code     300259 non-null  object        
 3   num_code    300259 non-null  object        
 4   dep_time    300259 non-null  object        
 5   from        300259 non-null  object        
 6   time_taken  300259 non-null  object        
 7   stop        300259 non-null  object        
 8   arr_time    300259 non-null  object        
 9   to          300259 non-null  object        
 10  price       300259 non-null  int64         
 11  type        300259 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(10)
memory usage: 27.5+ MB


In [19]:
# I went through each column's unique values and their value_counts()
# Found out that 'stop' column has lot of garbage values as below
raw_data['stop'].value_counts()

stop
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\t\t\t\t                      243601
non-stop                                                                         36044
2+-stop                                                                          13288
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IXU\n\t\t\t\t\t\t\t\t\t\t\t\t                 1839
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia IDR\n\t\t\t\t\t\t\t\t\t\t\t\t                 1398
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Patna\n\t\t\t\t\t\t\t\t\t\t\t\t                674
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Indore\n\t\t\t\t\t\t\t\t\t\t\t\t               381
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia PAT\n\t\t\t\t\t\t\t\t\t\t\t\t                  354
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia MYQ\n\t\t\t\t\t\t\t\t\t\t\t\t                  321
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia Bhubaneswar\n\t\t\t\t\t\t\t\t\t\t\t\t          301
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia KLH\n\t\t\t\t\t\t\t\t\t\t\t\t                  284
1-stop\n\t\t\t\t\t\t\t\t\t\t\t\tVia JG

In [20]:
# We will replace all values starting with 1-stop to '1-stop' 
raw_data['stop'] = raw_data['stop'].apply(lambda x: '1-stop' if x.startswith('1-stop') else x )
print('After preprocessing: ')
raw_data['stop'].value_counts()

After preprocessing: 


stop
1-stop       250927
non-stop      36044
2+-stop       13288
Name: count, dtype: int64

##### Now, the preprocessed data is ready for exploration. Lets export data to './data/processed/' folder as 'processed.csv'

In [21]:
raw_data

,date,airline,ch_code,num_code,dep_time,from,time_taken,stop,arr_time,to,price,type
0,2022-02-11,SpiceJet,SG,8709,18:55,Delhi,02h 10m,non-stop,21:05,Mumbai,5953,Economy
1,2022-02-11,SpiceJet,SG,8157,06:20,Delhi,02h 20m,non-stop,08:40,Mumbai,5953,Economy
2,2022-02-11,AirAsia,I5,764,04:25,Delhi,02h 10m,non-stop,06:35,Mumbai,5956,Economy
3,2022-02-11,Vistara,UK,995,10:20,Delhi,02h 15m,non-stop,12:35,Mumbai,5955,Economy
4,2022-02-11,Vistara,UK,963,08:50,Delhi,02h 20m,non-stop,11:10,Mumbai,5955,Economy
...,...,...,...,...,...,...,...,...,...,...,...,...
300254,2022-03-31,Vistara,UK,822,09:45,Chennai,10h 05m,1-stop,19:50,Hyderabad,69265,Business
300255,2022-03-31,Vistara,UK,826,12:30,Chennai,10h 25m,1-stop,22:55,Hyderabad,77105,Business
300256,2022-03-31,Vistara,UK,832,07:05,Chennai,13h 50m,1-stop,20:55,Hyderabad,79099,Business
300257,2022-03-31,Vistara,UK,828,07:00,Chennai,10h 00m,1-stop,17:00,Hyderabad,81585,Business


In [22]:
processed_data_folder = "../data/processed"

raw_data.to_csv(processed_data_folder+"/processed.csv", index=False)
print('Processed file exported.')
print('List of files under ../data/processed: {0}'.format(os.listdir(processed_data_folder)))

Processed file exported.
List of files under ../data/processed: ['.gitkeep', 'processed.csv']
